In [79]:
#
# Public elliptic curve (sec512r1)

p = 2 ** 521 - 1
a = 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFC
b = 0x0051953EB9618E1C9A1F929A21A0B68540EEA2DA725B99B315F3B8B489918EF109E156193951EC7E937B1652C0BD3BB1BF073573DF883D2C34F1EF451FD46B503F00
E = EllipticCurve(FiniteField(p), [a, b])

x = 0x00C6858E06B70404E9CD9E3ECB662395B4429C648139053FB521F828AF606B4D3DBAA14B5E77EFE75928FE1DC127A2FFA8DE3348B3C1856A429BF97E7E31C2E5BD66
y = 0x011839296A789A3BC0045C8A5FB42C7D1BD998F54449579B446817AFBD17273E662C97EE72995EF42640C550B9013FAD0761353C7086A272C24088BE94769FD16650
G = E(x, y)

n = 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFA51868783BF2F966B7FCC0148F709A5D03BB5C9B8899C47AEBB6FB71E91386409

Fn = FiniteField(n)
print(int(n).bit_length())

(n * G).is_zero()

521


True

In [80]:
import os
import json
import base64
import hashlib

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac

#
# Common functions

def i2b(i):
    """
    int to bytes
    """
    return int(i).to_bytes((int(i).bit_length() + 7) // 8, byteorder=sys.byteorder)

def b2i(b):
    """
    bytes to int
    """
    return int.from_bytes(b, byteorder=sys.byteorder)

def ecp2b(p):
    """
    elliptic curve point to bytes
    """
    x, y = p.xy()
    return i2b(int(x)) + i2b(int(y))

def h(m):
    """
    The H function (SHA-256)
    """
    return hashlib.sha256(m).digest()

def hp(m):
    """
    The H' function
    """
    return int.from_bytes(h(m), byteorder=sys.byteorder) * G

def auth_enc(key, message):
    
    # pad message with 0 as described in 3.1.1.
    message += b'\x00' * 16
    #while len(message) % 16 != 0:
    #    message += b'\x00'
    
    # iv should be 0 according to RFC
    iv = b'\x00' * 12  # os.urandom(12)
    
    # return iv and cipher
    return AESGCM(key).encrypt(iv, message, None)

def auth_dec(key, cipher):
    
    # iv should be 0 according to RFC
    iv = b'\x00' * 12  # os.urandom(12)
    
    # return message after removing the last 16 "0" bytes
    return AESGCM(key).decrypt(iv, cipher, None)[:-16]

def e(pub, id, ssid):
    return h(pub, id, ssid)

def key_ex_s(p_s, x_s, X_s, P_u, X_u, id_s, id_u, ssid):
    
    e_u = h(ecp2b(X_u) + id_s + ssid)
    e_s = h(ecp2b(X_s) + id_u + ssid)
    
    return h(ecp2b((X_u + b2i(e_u) * P_u) * (x_s + b2i(e_s) * p_s)))

def key_ex_u(p_u, x_u, X_s, P_s, X_u, id_s, id_u, ssid):
    
    e_u = h(ecp2b(X_u) + id_s + ssid)
    e_s = h(ecp2b(X_s) + id_u + ssid)
    
    return h(ecp2b((X_s + b2i(e_s) * P_s) * (x_u + b2i(e_u) * p_u)))

def f(key, message):
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(message)
    return h.finalize()

def abort():
    sys.exit(-1)
    None

#k = AESGCM.generate_key(bit_length=int(128))
#c = auth_enc(k, b'hello')
#print(c)
#print(auth_dec(k, c))

In [81]:
#
# Step 1. User

# choose password
pw = b'pwd123'

# choose private and public key
prv_u = Integer(Fn.random_element())
pub_u = prv_u * G

In [82]:
#
# Step 2. Server

# choose random key for OPRF (different for each user)
k_u = Integer(Fn.random_element())
v_u = k_u * G

# choose private and public key
prv_s = Integer(Fn.random_element())
pub_s = prv_s * G

In [83]:
# Server sends pub_s to user

In [84]:
#
# OPRF(k_u; pw) between server and user

In [85]:
#
# Step 3.1 User

# choose random r
r = Integer(Fn.random_element())

# compute alpha
alpha = hp(pw) + r * G

In [86]:
# User sends alpha to server

In [87]:
#
# Step 3.2. Server

# compute beta
beta = k_u * alpha

In [88]:
# Serveur sends v_u and beta to user

In [89]:
#
# Step 3.3. User

# compute rw
v_u_x, v_u_y = v_u.xy()
rw = h(pw + ecp2b(beta + -r * v_u))
print(rw)

b'\xcf\x17n4\xac\xe9\xf6\xe2-\x13\x04\tQ\xb3\xe6z\x1f\x13\x95\x19\xf83\x8d\x81\xa45\x17\x82\xe9\xa8\tt'


In [90]:
#
# Continue password registration

In [124]:
#
# Step 3. User

# compute c

import pickle

data = {
    'p_u': prv_u,
    'P_u': pub_u,
    'P_s': pub_s
}

ser_data = pickle.dumps((prv_u, pub_u, pub_s))
c = auth_enc(rw, ser_data)

In [92]:
# User sends c and pub_u to server

In [125]:
# Server stores <c, pub_s, prv_s, pub_u, k_u, v_u> for the corresponding user
# User must erase pw, rw and all keys

print("c =", c)

x, y = pub_s.xy()
print("pub_s = E({}, {})".format(x, y))
#print("prv_s =", prv_s)

x, y = pub_u.xy()
print("pub_u = E({}, {})".format(x, y))
#print("k_s =", k_u)

c = b'C#\xf4z\xcf_\xaf\x84\xf6-\x08\xd7h\x16\xa0R\x81x\n\xbf\xd0\x9ak^\xb7\xfc\x856\x11\xb4~\xda\xb2g\xae\x83\x95t-\xa7\x83&@@\xca\xcc\xc9=@\x96\x84\x06%\x8d\x11|\xe1\x94\xfe\x7f\x9fu\x0e\xbb\x05\x93\xf4\xbf\xba3\x01x\x80\xd8d\x1c\xb3\x98\x14\xa9\xa2\xb6uC\xc5jE\n\xd6\xb4\x1b\x00\xd5\xa8\r\x1a\x98\xce\x82\xe7ip+\xf9\xb0,\xc7\xa9P3\xf6\x03`\x15py\xa6\xfbI\xee[\xb7\x1b\xad\xe6S\xa2\x02B\x85K\xe0\xa1\x9d\n\x95\xc7X\x1bm\xe3\xfd:Cs\xcb\xc0\xf1^\xe55/n\xdf\x1a_0\x19oD\n\xa5\xe2.\x9b\xbc\x8aO\xd18\\\xa7\xc9\x03\x03\xb3\xbe\xedU8\x84[\xec\xb0\x99\x8d`\xbe\xdbN6\xdd%\'h}m\x1f\xbe\x99\xfc\x92\x84?\xab\xa02\x118VPY\xc3\x84tv\xd3\xd7i<a\xa6*Q2\xa2\x0b\x02\xdb\x93\x1f\xd0G8(\x1f\x9f\x1c\x1c5x<N\x98\xfe]\x04\x8b\x9e\x9c\x12\xffsU%f\xff\x84z\xac\x08\xb1\xb8+\t\'\xdcvs\x89\xf5\x06?\x82\xafEC\x10\xdf\xfe\xf08E\xbb\xda\x02x?\x06\x88\xbb\xdf\x88\xf1mQ\x947\xce\xdf\xd6.\xb1E\xc5WL<\x8d\x1f\xda)\x8c!d\xbeW7{\x81\x1b\xcc\xc4\xcd|\x14\xbd-BR` \x19i4vah\x8f\x98\xf3\x1cJ\xeeI\xab\xb0\xb7\x97\x90\x02\xf8V{.\xc